In [147]:
import re
import string
from sklearn.model_selection import train_test_split
from nltk.util import ngrams,everygrams
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.lm import Vocabulary, MLE
# import pandas as pd
from numpy.random import multinomial as multi
from random import randrange

# from keras.layers import LSTM

## Read, Parse and Divide Trump speech data

In [163]:
file=open("trumpspeeches.txt",'r')
text=file.read()

In [164]:
list_sent=list(sent_tokenize(text))

In [165]:
for sent in list_sent:
    sent=sent.replace('\n'," ")
    sent=sent.replace('-'," ")
    sent=sent.translate(str.maketrans('', '', string.punctuation))
    sent=re.sub(r"\d",'',sent)
    sent=sent.replace("SPEECH",'')
    sent=sent.replace('  '," ")
    sent=sent.lower()
    list_sent.pop(0)
    list_sent.append(sent)

In [166]:
list_sent[0]

'  thank you so much'

In [167]:
text_train, text_test=train_test_split(list_sent,test_size=0.2,train_size=0.8, random_state=42)
print(len(text_train))
print(len(text_test))

13194
3299


In [168]:
text_train

['but i don’t know',
 'think of it',
 'the iran deal like so many of our worst agreements is the result of not being willing to leave the table',
 'it’s going to happen',
 'his daughter works for hillary clinton',
 'but most people aren’t and they believe something',
 'would i bring my children into the administration',
 'they don’t believe a lot of what they see in the media because if you believe these people – you know why are they doing this',
 'and they set it back years',
 'pfizer great company pharmaceutical company they’re going to ireland',
 'you saw in nevada i won in the poll the hispanic vote',
 'and they were comparing it',
 'and it takes courage to run',
 'i think the press – some of the press said i did fantastically well',
 'no no they’re so dishonest',
 'now i know some people say i’m too much of a fighter',
 'a lot of good things and that’s going to continue because they’re not going to be able to do it folks',
 'he deserted',
 'they have to end it and now',
 'why are

## Q1

In [169]:
def gettoken(sent_tokens,if_sent=False):
    tokens=[]
    if(if_sent==False):
        for i in range(len(sent_tokens)):
            temp=word_tokenize(sent_tokens[i])
            tokens.append('<s>')
            tokens.extend(temp)
            tokens.append('</s>')
    else:
        temp=word_tokenize(sent_tokens)
        tokens.append('<s>')
        tokens.extend(temp)
        tokens.append('</s>')
    return tokens

In [170]:
vocab=[]
train_tokens=gettoken(text_train)
vocab=np.unique(train_tokens)
output=[]
for i in range(len(text_test)):
    output.append(gettoken(text_test[i],if_sent=True))
# vocab=np.unique(np.concatenate((vocab,np.array(token1)), axis=None))
# print(len(vocab))

In [171]:
train_tokens

['<s>',
 'but',
 'i',
 'don',
 '’',
 't',
 'know',
 '</s>',
 '<s>',
 'think',
 'of',
 'it',
 '</s>',
 '<s>',
 'the',
 'iran',
 'deal',
 'like',
 'so',
 'many',
 'of',
 'our',
 'worst',
 'agreements',
 'is',
 'the',
 'result',
 'of',
 'not',
 'being',
 'willing',
 'to',
 'leave',
 'the',
 'table',
 '</s>',
 '<s>',
 'it',
 '’',
 's',
 'going',
 'to',
 'happen',
 '</s>',
 '<s>',
 'his',
 'daughter',
 'works',
 'for',
 'hillary',
 'clinton',
 '</s>',
 '<s>',
 'but',
 'most',
 'people',
 'aren',
 '’',
 't',
 'and',
 'they',
 'believe',
 'something',
 '</s>',
 '<s>',
 'would',
 'i',
 'bring',
 'my',
 'children',
 'into',
 'the',
 'administration',
 '</s>',
 '<s>',
 'they',
 'don',
 '’',
 't',
 'believe',
 'a',
 'lot',
 'of',
 'what',
 'they',
 'see',
 'in',
 'the',
 'media',
 'because',
 'if',
 'you',
 'believe',
 'these',
 'people',
 '–',
 'you',
 'know',
 'why',
 'are',
 'they',
 'doing',
 'this',
 '</s>',
 '<s>',
 'and',
 'they',
 'set',
 'it',
 'back',
 'years',
 '</s>',
 '<s>',
 'pfizer

In [172]:
def getgram(n):
    grams = list(everygrams(train_tokens,max_len=n))
    mle = MLE(n)
    mle.fit([grams], vocabulary_text=vocab)
    return mle
for i in range(1,5):
    print("No of ",i,"-gram Theoretically possible: ",len(vocab)**i)
    print("No of ",i,"-gram Practically: ",len(set(ngrams(train_tokens,i))))
unigram=getgram(1)
bigram=getgram(2)
trigram=getgram(3)
quadgram=getgram(4)
# print("lo")

No of  1 -gram Theoretically possible:  4325
No of  1 -gram Practically:  4325
No of  2 -gram Theoretically possible:  18705625
No of  2 -gram Practically:  28489
No of  3 -gram Theoretically possible:  80901828125
No of  3 -gram Practically:  53081
No of  4 -gram Theoretically possible:  349900406640625
No of  4 -gram Practically:  74306


In [173]:
unigram

In [174]:
def generate(n,model):
    start="<s>"
    gen_sent=[start]
    if(n==1):
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
    elif(n==2):
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i,[start]) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
    elif(n==3):
        gen=[start]
        index=np.argmax(multi(1,[model.score(i,[start]) for i in vocab]))
        gen_sent.append(vocab[index])
        gen.append(vocab[index])
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i,gen) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
            gen=gen[1:]
    elif(n==4):
        gen=[start]
        index=np.argmax(multi(1,[model.score(i,[start]) for i in vocab]))
        gen_sent.append(vocab[index])
        gen.append(vocab[index])
        index=np.argmax(multi(1,[model.score(i,gen) for i in vocab]))
        gen_sent.append(vocab[index])
        gen.append(vocab[index])
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i,gen) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
            gen=gen[1:]
    return ' '.join(gen_sent)
print("Unigram:")
for i in range(10):
    print(generate(1,unigram))
print("\nBigram:")
for i in range(10):
    print(generate(2,bigram))
print("\nTrigram:")
for i in range(10):
    print(generate(3,trigram))
print("\nQuadgram:")
for i in range(10):
    print(generate(4,quadgram))

Unigram:
<s> deals i it country know <s> vets to you the millions days he taking we like i shut to that they believed making he million up one a a ever with ’ have we </s>
<s> <s> well have be of you going ’ the i and really in <s> and in libya ’ ’ world our one bad ’ thats ’ and who groups go approved since there totally line <s> congress showed things off a of ’ re – crowds raised never called words <s> a </s>
<s> we want at </s>
<s> <s> nobody he economy t policy ’ <s> some stupid you the into </s>
<s> deal <s> <s> majority tell i but </s>
<s> just for pearl <s> <s> going ’ much these ’ you said by ve say many discuss be it it i very now so </s>
<s> to </s>
<s> home things re <s> preparedness me guy will snail – and a ’ never funding </s>
<s> beautiful it </s>
<s> </s>

Bigram:
<s> so i can ’ ll be running </s>
<s> i learned that </s>
<s> oh but look if i take me a win a nasty </s>
<s> but i have to look i do is happening </s>
<s> and thousands of religion of many great scams </s>
<

In [175]:
from random import randrange
tes=[randrange(0,len(output),1) for j in range(10)]
def perplexity(models):
    for i in tes:
        print("PP of ",' '.join(output[i]))
        for j in range(len(models)):
            mle=models[j]
            try:
                print(j+1," Gram ",round(mle.perplexity(ngrams(output[i],j+1)),2))
            except:
                print(j+1," Gram ","inf")
models=[unigram,bigram,trigram,quadgram]
perplexity(models)

PP of  <s> we knocked the hell out of iraq </s>
1  Gram  289.91
2  Gram  31.3
3  Gram  6.35
4  Gram  1.41
PP of  <s> but if she ’ s allowed to run honestly it ’ ll be a sad day for this country because what she did was wrong </s>
1  Gram  282.67
2  Gram  inf
3  Gram  inf
4  Gram  inf
PP of  <s> we ’ re going to win </s>
1  Gram  56.71
2  Gram  5.0
3  Gram  2.79
4  Gram  2.44
PP of  <s> this you ’ ll get a kick out of </s>
1  Gram  inf
2  Gram  inf
3  Gram  inf
4  Gram  inf
PP of  <s> franklin graham came out with the most beautiful statements </s>
1  Gram  782.63
2  Gram  14.41
3  Gram  2.98
4  Gram  1.17
PP of  <s> and we talk about my tone </s>
1  Gram  197.14
2  Gram  24.67
3  Gram  9.16
4  Gram  3.55
PP of  <s> they ’ re political hacks </s>
1  Gram  153.49
2  Gram  10.8
3  Gram  5.74
4  Gram  3.84
PP of  <s> i mean you give money and they get a cut </s>
1  Gram  148.27
2  Gram  inf
3  Gram  inf
4  Gram  inf
PP of  <s> i gave tremendous amounts of money away </s>
1  Gram  359.13
2 

Quadgrams are more readable than tri, which are more readable than bi which are more readable than uni